In [5]:
import data
import eval
import model
import preprocess
import postprocess

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from sklearn.model_selection import train_test_split


ONLY RUN IF THERE IS NEW DATA, THIS TAKES LONG, NEXT CELL LOADS DATA FROM CSV AND PKL FILE

First off, the steps taken to fully load in the data set.

Step 1: load the raw text files and their id's into a dataframe

Step 2: if the texts are not in English, translate them

Step 3: concatenate the text files into a single dataset

Step 4: load in the labels

Step 5: merge the text and labels data sets

This can take quite a while, due to the translation step, skip this for the next cell, once we have all the data correctly saved as a csv, so we just load in that csv again. 

In [12]:
# load english text data
df_text_en = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/raw-documents")

# load and translate hindi and portugese text - part that takes long
df_text_bg = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/raw-documents", True, source_lang ="bg")
df_text_hi = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/raw-documents", True, source_lang="hi")
df_text_pt = data.load_text_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/raw-documents", True, source_lang = "pt")
df_text = pd.concat([df_text_en, df_text_bg, df_text_hi, df_text_pt])

# load all the labels, these are all in english
df_labels_en = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-2-annotations.txt")
df_labels_bg = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/subtask-2-annotations.txt")
df_labels_hi = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/subtask-2-annotations.txt")
df_labels_pt = data.load_labels("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/subtask-2-annotations.txt")
df_labels = pd.concat([df_labels_en, df_labels_bg, df_labels_hi, df_labels_pt])


dom_mlb, sub_mlb, df = preprocess.encode_labels(pd.merge(df_text, df_labels, on="id"))

# save the mlb's, and the dataframe, so we can read them instead of processing the entire data again
with open("dom_mlb.pkl", "wb") as f:
    pickle.dump(dom_mlb,f)

with open("sub_mlb.pkl", "wb") as f:
    pickle.dump(sub_mlb,f)

df.to_csv("data.csv")

Data length:  200
Error while reading:BG_828.txt
Error while reading:BG_551.txt
Error while reading:BG_1559.txt
Error while reading:BG_630.txt


KeyboardInterrupt: 

first load the data set, and save it as a csv?

In [3]:
df = pd.read_csv("data.csv")

# load the mlb files back in, to get the classes and transform functions
with open("dom_mlb.pkl", "rb") as f:
    dom_mlb = pickle.load(f)

with open("sub_mlb.pkl", "rb") as f:
    sub_mlb = pickle.load(f)

In [4]:
# add this to the df creator above? 
from sentence_transformers import SentenceTransformer
bertmodel = SentenceTransformer("all-mpnet-base-v2")
df["SBERTembeddings"] = df["text"].apply(lambda x: preprocess.embed_x(x,bertmodel))

In [9]:
df.to_clipboard()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
df_explanation_en = data.load_explanation_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/EN/subtask-3-annotations.txt")
df_explanation_bg = data.load_explanation_data("/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/BG/subtask-3-annotations.txt")
df_explanation_hi = data.load_explanation_data('/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/HI/subtask-3-annotations.txt')
df_explanation_pt = data.load_explanation_data('/Users/jochem/Documents/school/Uni KI jaar 4/Scriptie/Train Data/training_data_16_October_release/PT/subtask-3-annotations.txt')

explanation_df = pd.concat([df_explanation_en, df_explanation_bg, df_explanation_hi, df_explanation_pt])

df = pd.merge(df, explanation_df, on="id")

   Unnamed: 0                id  \
0           2  EN_UA_011260.txt   
1           6  EN_CC_100047.txt   
2          14  EN_UA_026142.txt   
3          17  EN_CC_100124.txt   
4          21  EN_CC_100050.txt   

                                                text  \
0  Russia has a clear plan to resolve the conflic...   
1  Biden Has Omnibus Bill, Packed With Climate Ch...   
2  Oppose imperialist War! SEP/IYSSE public meeti...   
3  Glaciers Give the Lie to Global Warming Narrat...   
4  United Nations COP28 ‘Climate’ Summit Takes Ai...   

                                            dom_narr  \
0  ['URW: Russia is the Victim', 'URW: Discrediti...   
1  ['CC: Criticism of institutions and authoritie...   
2  ['URW: Amplifying war-related fears', 'URW: Am...   
3  ['CC: Downplaying climate change', 'CC: Downpl...   
4  ['CC: Criticism of institutions and authoritie...   

                                            sub_narr  \
0  ['URW: Russia is the Victim: UA is anti-RU ext...   
1  

Train it on the sub classes

In [52]:
vec = TfidfVectorizer(min_df=0.01,max_df=0.35)
X = vec.fit_transform(df.text)
y = df[sub_mlb.classes_].values # omg this works
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [53]:
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=sub_mlb.classes_))

Skipped fold 51, labels invalid
Skipped fold 80, labels invalid
Skipped fold 89, labels invalid
Skipped fold 145, labels invalid
Skipped fold 165, labels invalid
Skipped fold 178, labels invalid
Skipped fold 204, labels invalid
Skipped fold 214, labels invalid
Skipped fold 229, labels invalid
Skipped fold 356, labels invalid
Skipped fold 427, labels invalid
Skipped fold 428, labels invalid
Skipped fold 562, labels invalid
                                                                                                            precision    recall  f1-score   support

                                 CC: Amplifying Climate Fears: Amplifying existing fears of global warming       0.64      0.71      0.67        45
                                               CC: Amplifying Climate Fears: Doomsday scenarios for humans       0.32      0.54      0.40        13
                                            CC: Amplifying Climate Fears: Earth will be uninhabitable soon       0.50      0.56  

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Trained on the dom classes:

In [12]:
vec = TfidfVectorizer(min_df=0.01,max_df=0.35)
X = vec.fit_transform(df.text)
y = df[dom_mlb.classes_].values
y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)
print(classification_report(y_true = y_true, y_pred=y_pred, target_names=dom_mlb.classes_))

                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.65      0.86      0.74        70
                      CC: Climate change is beneficial       0.00      0.00      0.00         3
              CC: Controversy about green technologies       0.20      0.10      0.13        10
                     CC: Criticism of climate movement       0.45      0.43      0.44        23
                     CC: Criticism of climate policies       0.46      0.71      0.56        38
         CC: Criticism of institutions and authorities       0.51      0.75      0.60        56
                        CC: Downplaying climate change       0.50      0.40      0.44        15
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         2
 CC: Hidden plots by secret schemes of powerful groups       0.50      0.09      0.15        11
          CC: Questioning the measureme

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Without loocv and majority classifier:

In [ ]:
vec = TfidfVectorizer(min_df=0.01,max_df=0.35)
X = vec.fit_transform(df.text)

y = df[dom_mlb.classes_].values


X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.1, random_state = 1)
y_pred = model.logistic_regression_classifier(X_train=X_train, y_train=y_train,X_test=X_test)

print(classification_report(y_test,y_pred,target_names=dom_mlb.classes_, zero_division=0.0))

                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.55      0.86      0.67        14
                      CC: Climate change is beneficial       0.00      0.00      0.00         1
              CC: Controversy about green technologies       0.00      0.00      0.00         2
                     CC: Criticism of climate movement       1.00      0.38      0.55         8
                     CC: Criticism of climate policies       0.89      0.67      0.76        12
         CC: Criticism of institutions and authorities       0.45      0.69      0.55        13
                        CC: Downplaying climate change       0.67      0.50      0.57         4
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         0
 CC: Hidden plots by secret schemes of powerful groups       0.00      0.00      0.00         4
          CC: Questioning the measureme

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:

X = np.vstack(df["SBERTembeddings"].values)
print(X.shape)
print(X[1].shape)
print(X[2].shape)
y = df[dom_mlb.classes_].values

y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)

print(classification_report(y_true=y_true,y_pred=y_pred,target_names=dom_mlb.classes_))

(626, 384)
(384,)
(384,)
                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.52      0.90      0.66        70
                      CC: Climate change is beneficial       0.08      1.00      0.15         3
              CC: Controversy about green technologies       0.16      0.90      0.27        10
                     CC: Criticism of climate movement       0.21      0.96      0.35        23
                     CC: Criticism of climate policies       0.25      0.97      0.39        38
         CC: Criticism of institutions and authorities       0.35      1.00      0.52        56
                        CC: Downplaying climate change       0.15      0.93      0.25        15
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         2
 CC: Hidden plots by secret schemes of powerful groups       0.12      0.91      0.21        11
          CC: 

In [5]:
X = np.vstack(df["SBERTembeddings"].values)
print(X.shape)
print(X[1].shape)
print(X[2].shape)
y = df[dom_mlb.classes_].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y_pred = model.logistic_regression_classifier(X_train=X_train,y_train=y_train,X_test=X_test)

pd.DataFrame(y_pred).to_clipboard()


print(classification_report(y_true=y_test,y_pred=y_pred,target_names=dom_mlb.classes_))

(626, 384)
(384,)
(384,)
                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.50      0.80      0.62        15
                      CC: Climate change is beneficial       0.00      0.00      0.00         0
              CC: Controversy about green technologies       0.18      1.00      0.31         2
                     CC: Criticism of climate movement       0.28      1.00      0.43         5
                     CC: Criticism of climate policies       0.24      1.00      0.39         7
         CC: Criticism of institutions and authorities       0.39      1.00      0.56        11
                        CC: Downplaying climate change       0.17      1.00      0.29         3
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         0
 CC: Hidden plots by secret schemes of powerful groups       0.08      1.00      0.14         1
          CC: 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
X = np.vstack(df["SBERTembeddings"].values)
print(X.shape)
print(X[1].shape)
print(X[2].shape)
y = df[dom_mlb.classes_].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

y_pred = model.logistic_regression_classifier(X_train=X_train,y_train=y_train,X_test=X_test)

pd.DataFrame(y_pred).to_clipboard()


print(classification_report(y_true=y_test,y_pred=y_pred,target_names=dom_mlb.classes_))

(626, 768)
(768,)
(768,)
                                                        precision    recall  f1-score   support

                          CC: Amplifying Climate Fears       0.50      0.85      0.63        13
                      CC: Climate change is beneficial       0.00      0.00      0.00         0
              CC: Controversy about green technologies       0.20      0.67      0.31         3
                     CC: Criticism of climate movement       0.28      1.00      0.43         5
                     CC: Criticism of climate policies       0.35      1.00      0.52         8
         CC: Criticism of institutions and authorities       0.38      1.00      0.56        10
                        CC: Downplaying climate change       0.12      1.00      0.21         2
       CC: Green policies are geopolitical instruments       0.00      0.00      0.00         0
 CC: Hidden plots by secret schemes of powerful groups       0.14      1.00      0.25         2
          CC: 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
vec = TfidfVectorizer(min_df=0.01,max_df=0.35)
X = vec.fit_transform(df.text)
y = df[sub_mlb.classes_].values

y_pred, y_true = model.logistic_regression_loocv(X,y,majority_ensemble=True)

print(classification_report(y_true=y_true,y_pred=y_pred,target_names=sub_mlb.classes_))

Skipped fold 51, labels invalid
Skipped fold 80, labels invalid
Skipped fold 89, labels invalid
Skipped fold 145, labels invalid
Skipped fold 165, labels invalid
Skipped fold 178, labels invalid
Skipped fold 204, labels invalid
Skipped fold 214, labels invalid
Skipped fold 229, labels invalid
Skipped fold 356, labels invalid
Skipped fold 427, labels invalid
Skipped fold 428, labels invalid
Skipped fold 562, labels invalid
                                                                                                            precision    recall  f1-score   support

                                 CC: Amplifying Climate Fears: Amplifying existing fears of global warming       0.64      0.71      0.67        45
                                               CC: Amplifying Climate Fears: Doomsday scenarios for humans       0.32      0.54      0.40        13
                                            CC: Amplifying Climate Fears: Earth will be uninhabitable soon       0.50      0.56  

/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jochem/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
